# NSIDC DAAC Customize and Access Data Tutorial

This tutorial will walk you though how to access NSIDC DAAC data using spatial and temporal filters, as well as how to request customization services including subsetting, reformatting, and reprojection. 


Import packages


In [2]:
# put necessary imports here

import requests
import getpass
import socket
import pycurl
import urllib.request
import json


## Create a token

We will generate a token needed in order to access data using your Earthdata Login credentials, and we will apply that token to the following queries.

In [3]:
# Earthdata Login credentials

uid = input('Earthdata Login user name: ')
pswd = getpass.getpass('Earthdata Login password:')


Earthdata Login user name: amy.steiker
Earthdata Login password:········


In [4]:
# Function used to request token from Common Metadata Repository

def create_token (uid, pswd):
    
    #import XML Element Tree
    from xml.etree.ElementTree import Element, SubElement, Comment, tostring
    
    #Find IP address
    hostname = socket.gethostname() 
    IP = socket.gethostbyname(hostname)
    
    #Create XML input

    token = Element('token')
    username = SubElement(token, 'username')
    username.text = uid
    password = SubElement(token, 'password')
    password.text = pswd
    client_id = SubElement(token, 'client_id')
    client_id.text = 'NSIDC_client_id'
    user_ip_address = SubElement(token, 'user_ip_address')
    user_ip_address.text = IP

    xml = (tostring(token, encoding='unicode', method='xml'))
    
    #Request token from Common Metadata Repository

    headers = {'Content-Type': 'application/xml'} 
    token = requests.post('https://api.echo.nasa.gov/echo-rest/tokens', data=xml, headers=headers)
    output = token.text
    
    #Grab token string

    start = '<id>'
    end = '</id>'

    tokenval =(output.split(start))[1].split(end)[0]
    
    return tokenval

In [5]:
# Run create_token function using Earthdata Login Username (uid) and Password (pswd) as inputs

token = create_token(uid, pswd)
print(token)

8AA1D7B8-DA57-D9DD-AE37-99AC1E11C669


## Determine the services enabled for your data set of interest.

We will do this by first inputting the data set of interest and the most recent version number.

In [116]:
# Input data set short name (e.g. ATL03) of interest here.

short_name = input('Input short name, e.g. ATL03, here: ')


Input short name, e.g. ATL03, here: SPL3SMP


In [117]:
# For restricted collections
# First determine the latest verion number by querying CMR collection metadata.


# mr_url = 'https://cmr.earthdata.nasa.gov/search/collections.json?short_name=' + short_name + '&token=' + token
# cmeta = requests.get(cmr_url)
 
# with urllib.request.urlopen(cmr_url) as url:
#    cmeta_json = json.loads(url.read().decode())

Let's determine the most recent verion number by querying CMR collection metadata.


In [128]:
# Get json response from CMR collection metadata


cmr_url = 'https://cmr.earthdata.nasa.gov/search/collections.json?short_name=' + short_name
cmeta = requests.get(cmr_url)
 
with urllib.request.urlopen(cmr_url) as url:
    cmeta_json = json.loads(url.read().decode())

# Find all instances of 'version_id' in metadata and print most recent version number
    
entry = cmeta_json['feed']['entry']
versions = []
for x in range(len(entry)):
    versions.append((entry[x]['version_id']))

latest_version = max(versions)
print('The most recent version of ', short_name, ' is ', latest_version)
    

The most recent version of  SPL3SMP  is  005


Now let's find the services available for this data set and version.

In [ ]:
# Input version of interest here

# https://n5eil02u.ecs.nsidc.org/egi/capabilities/ATL03.205.xml

## Download data

In [15]:
# Download data test (still in progress)

# base URL: https://n5eil02u.ecs.nsidc.org/egi/request?
# short_name=GLAH12
# version=034
# bounding_box=-50.33333,68.56667,-49.33333,69.56667
# bbox=-50.33333,68.56667,-49.33333,69.56667
# time=2009-01-01T00:00:00,2009-12-31T23:59:59
# format=TABULAR_ASCII
# token=TOKEN-FROM-STEP-2

access_url = 'https://n5eil02u.ecs.nsidc.org/egi/request?' + 'short_name=GLAH12&' + 'version=034&' + 'bounding_box=-50.33333,68.56667,-49.33333,69.56667&' + 'bbox=-50.33333,68.56667,-49.33333,69.56667&' + 'time=2009-01-01T00:00:00,2009-12-31T23:59:59&' + 'format=TABULAR_ASCII&' + 'token=' + token
print(access_url)

https://n5eil02u.ecs.nsidc.org/egi/request?short_name=GLAH12&version=034&bounding_box=-50.33333,68.56667,-49.33333,69.56667&bbox=-50.33333,68.56667,-49.33333,69.56667&time=2009-01-01T00:00:00,2009-12-31T23:59:59&format=TABULAR_ASCII&token=8AA1D7B8-DA57-D9DD-AE37-99AC1E11C669


In [20]:
import zipfile, io

In [21]:
r = requests.get(access_url, allow_redirects=True)
#open('google.ico', 'wb').write(r.content)
#resp = requests.get("http://www.example.com", 
                    #params = {"name":"Daniel", "id": 123456})
    
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()


#with open("data_output",'wb') as f: 
  
    # Saving received content as a png file in 
    # binary format 
  
    # write the contents of the response (r.content) 
    # to a new file in binary mode. 
    #f.write(r.content) 
  
